In [1]:
import yfinance as yf
import pandas as pd
import psycopg as pg
from daily_insert.config import DB_CONFIG

with pg.connect(**DB_CONFIG) as conn:
    with conn.cursor() as cur:
        cur.execute('SELECT DISTINCT(currency) FROM tickers')
        currency = cur.fetchall()

currency = [i[0] for i in currency]

pairs = []
for f in currency:
    pair = [c for c in currency if c != f]
    for p in pair:
        pairs.append(f'{f}{p}=X')

df_list = []
for pair in pairs:
    data = yf.download(pair, start='2020-01-01')
    data = data.droplevel(1, axis=1)
    data['currency_pair'] = f'{pair[:3]}/{pair[3:6]}'
    df_list.append(data)

df = pd.concat(df_list)

mask = (df['High'] < df['Open']) | (df['High'] < df['Close']) | (df['Low'] > df['Open']) | (df['Low'] > df['Close'])
clean = df[~mask].copy()
temp = df[mask].copy()

temp['High'] = temp[['Open', 'Close', 'High']].max(axis=1)
temp['Low'] = temp[['Open', 'Close', 'Low']].min(axis=1)
clean = pd.concat([clean, temp], axis=0)

clean = clean.reset_index()
clean.drop(columns=['Adj Close', 'Volume'], inplace=True)

from io import StringIO

buffer = StringIO()
clean.to_csv(buffer, index=False, header=False)
buffer.seek(0)

with pg.connect(**DB_CONFIG) as conn:
    # The COPY operation needs to be within a WITH block
    with conn.cursor().copy("COPY daily_forex (date, close, high, low, open, currency_pair) FROM STDIN WITH CSV") as copy:
        copy.write(buffer.getvalue())
    
    conn.commit()